# ==========================================
# 步驟 1: 安裝與設定環境
# ==========================================

In [ ]:
# ==========================================
# 步驟 1: 安裝與設定環境 (最終修正版)
# ==========================================

# 1. 強制安裝新版 SDK
!pip install -q -U --force-reinstall google-genai replicate "moviepy<2.0"

# --- 導入區塊 ---
import os
import json
import requests
import replicate
from google.colab import userdata

# [舊版 SDK] 用於文字生成 (保持您原有的習慣)
import google.generativeai as genai

# [新版 SDK] 用於圖片生成 (給它一個新別名 genai_new 以免衝突)
from google import genai as genai_new

# --- 導入 MoviePy ---
from moviepy.editor import ImageClip, AudioFileClip

# --- 設定金鑰 ---
try:
    # 讀取金鑰
    GEMINI_KEY = userdata.get('gemini_api_key')

    # 設定舊版 SDK (給文字生成用)
    genai.configure(api_key=GEMINI_KEY)

    # 設定 Replicate
    os.environ["REPLICATE_API_TOKEN"] = userdata.get('REPLICATE_API_TOKEN')

    print("✅ 成功讀取 Secrets！環境設定完成。")
except Exception as e:
    print(f"❌ 讀取 Secrets 失敗: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.6/317.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 76.8 MB/s eta 0:00:00
 

# ==========================================
# 步驟 2: 定義 AI Agent 功能
# ==========================================

In [ ]:
import json
import google.generativeai as genai

# 1. Gemini Agent (大腦 - 負責產出符合你風格的 Prompt)
def get_prompts_from_gemini(theme):
    print(f"🤖 Gemini 正在構思主題: {theme} (風格鎖定: 甜美/輕鬆/簡潔)...")
    model = genai.GenerativeModel('gemini-2.0-flash')

    # --- 關鍵修改：在 Prompt 中植入「風格守則 (Style Guide)」 ---
    prompt = f"""
    You are a creative director for a music video channel.
    Input Theme: '{theme}'.

    Your task is to generate THREE distinct prompts, but you MUST strictly adhere to the following STYLE GUIDE:

    1. **MUSIC STYLE (Relaxing & Cheerful)**:
       - Vibe: Happy, sweet, uplifting, and relaxing.
       - Instruments: Acoustic guitar, piano, light percussion, ukulele.
       - Tempo: Mid-tempo (80-100 BPM), not too slow, not too fast.
       - Avoid: Dark, sad, heavy metal, or intense electronic sounds.

    2. **VISUAL STYLE (Video & Image)**:
       - **Subject**: A sweet-looking young woman (approx 20 years old) with a gentle, charming smile. Beautiful and elegant.
       - **Composition**: Simple, clean, and uncluttered. Avoid complex backgrounds.
       - **Lighting**: Bright, soft natural sunlight, pastel tones, warm atmosphere.
       - **Action**: Very subtle and simple movements (e.g., smiling at the camera, hair blowing in the wind, holding a coffee).
       - **Quality**: Photorealistic, 8k, cinematic lighting, shallow depth of field (blurred background).

    Generate the output as a raw JSON object with these keys:
    1. 'music_prompt': Description for MusicGen.
    2. 'video_prompt': Description for video generation (focus on the woman and simple motion).
    3. 'image_prompt': Description for image generation (portrait focus).

    Return ONLY the JSON string:
    {{"music": "...", "video": "...", "image": "..."}}
    """

    try:
        response = model.generate_content(prompt)
        # 清理輸出
        cleaned_text = response.text.replace("```json", "").replace("```", "").strip()
        print(f"原始回應文本 (除錯用): \n{cleaned_text[:100]}...")

        data = json.loads(cleaned_text)
        final_prompts = {}

        # 容錯處理：確保抓到正確的 Key
        # Music
        if 'music' in data: final_prompts['music'] = data['music']
        elif 'music_prompt' in data: final_prompts['music'] = data['music_prompt']

        # Video
        if 'video' in data: final_prompts['video'] = data['video']
        elif 'video_prompt' in data: final_prompts['video'] = data['video_prompt']

        # Image
        if 'image' in data: final_prompts['image'] = data['image']
        elif 'image_prompt' in data: final_prompts['image'] = data['image_prompt']

        # 最終檢查
        if all(k in final_prompts for k in ['music', 'video', 'image']):
            print("✅ 風格化 Prompt 生成成功！")
            return final_prompts
        else:
            print(f"⚠️ 缺少 Key，目前只有: {final_prompts.keys()}")
            return None

    except json.JSONDecodeError:
        print("⚠️ JSON 解析失敗，請重試。")
        return None
    except Exception as e:
        print(f"⚠️ 發生錯誤: {e}")
        return None


# 2. Gemini Image (畫家 - 負責畫圖)
def generate_image_with_gemini(prompt, filename="gemini_image.png"):
    print(f"🎨 Gemini (Imagen 4) 正在繪圖...")

    try:
        # 使用新版 SDK 的 Client
        client = genai_new.Client(api_key=GEMINI_KEY)

        # 呼叫 Imagen API
        result = client.models.generate_images(
            model='imagen-3.0-generate-002', # 或是 'imagen-3.0-fast-generate-001'
            prompt=prompt,
            config=dict(
                number_of_images=1,
                output_mime_type="image/png",
                aspect_ratio="16:9",
            )
        )

        # 儲存圖片
        if result.generated_images:
            image_data = result.generated_images[0].image.image_bytes
            with open(filename, "wb") as f:
                f.write(image_data)
            print(f"✅ 圖片已儲存: {filename}")
            return filename
        else:
            print("❌ 圖片生成失敗：API 回應中未包含生成的圖片。")
            return None

    except Exception as e:
        print(f"❌ 圖片生成失敗: {e}")
        return None



# 3. Replicate API (音樂家 - 負責作曲)
def generate_music_api(prompt, duration=15, filename="api_music.wav"):
    print(f"🎵 Replicate (MusicGen Large) 正在作曲 ({duration}s)...")
    try:
        output = replicate.run(
            "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
            input={
                "prompt": prompt,
                "duration": duration,
                "model_version": "large"
            }
        )
        # 下載音樂檔案
        response = requests.get(output)
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"✅ 音樂已儲存: {filename}")
        return filename
    except Exception as e:
        print(f"❌ 音樂生成失敗: {e}")
        return None

# 4. 影片生成
def generate_video_api(prompt, filename="raw_video.mp4"):
    print(f"🎥 Replicate (Minimax) 正在生成影片...")
    try:
        # 呼叫模型
        output = replicate.run(
            "minimax/video-01",
            input={
                "prompt": prompt,
                "prompt_optimizer": True
            }
        )

        # --- 萬能下載邏輯 ---
        with open(filename, "wb") as f:
            # 情況 A: 如果是 FileOutput 物件
            if hasattr(output, "read"):
                f.write(output.read())

            # 情況 B: 如果是網址字串 (String)
            elif isinstance(output, str) and output.startswith("http"):
                f.write(requests.get(output).content)

            # 情況 C: 如果是清單 (List)
            elif isinstance(output, list) and len(output) > 0:
                item = output[0]
                if hasattr(item, "read"): # 清單裡是檔案物件
                    f.write(item.read())
                elif isinstance(item, str): # 清單裡是網址
                    f.write(requests.get(item).content)

            # 情況 D: 如果是生成器 (Iterator)
            elif hasattr(output, "__iter__"):
                item = next(iter(output))
                if hasattr(item, "read"):
                    f.write(item.read())
                elif isinstance(item, str):
                    f.write(requests.get(item).content)
            else:
                print(f"⚠️ 無法識別的回傳格式: {type(output)}")
                return None

        print(f"✅ 原始影片已下載: {filename}")
        return filename

    except Exception as e:
        print(f"❌ 影片生成失敗: {e}")
        try:
            print(f"除錯資訊 - Output Type: {type(output)}")
        except:
            pass
        return None


# 5. 影片合成
def assemble_video(image_path, audio_path, output_name="final_video.mp4"):
    if not image_path or not audio_path:
        return None
    print("🎬 正在合成影片...")
    try:
        audio_clip = AudioFileClip(audio_path)
        video_clip = ImageClip(image_path).set_duration(audio_clip.duration)
        video_clip = video_clip.set_audio(audio_clip)
        video_clip.write_videofile(output_name, fps=1, codec="libx264", audio_codec="aac", verbose=False, logger=None)
        print(f"🎉 影片製作完成！檔案名稱: {output_name}")
        return output_name
    except Exception as e:
        print(f"❌ 合成失敗: {e}")
        return None

# ==========================================
# 步驟 3: 執行生成
# ==========================================

In [ ]:
# 1. 取得 Prompt (記得要用新的 prompt 函式取得 video key)
# 設定您的主題
my_theme = "1920 年代美國鄉村風格"
prompts = get_prompts_from_gemini(my_theme)

# 2. 生成音樂 (假設你想要 20 秒)
music_file = generate_music_api(prompts['music'], duration=20)



🤖 Gemini 正在構思主題: 1920 年代美國鄉村風格 (風格鎖定: 甜美/輕鬆/簡潔)...
原始回應文本 (除錯用): 
{
  "music": "A relaxing and cheerful American folk song in the style of the 1920s. Features acousti...
✅ 風格化 Prompt 生成成功！
🎵 Replicate (MusicGen Large) 正在作曲 (20s)...
✅ 音樂已儲存: api_music.wav


In [ ]:
# 3. 生成影片 (使用 Replicate)
# 注意：prompts['video'] 是我們改過的 key，如果沒有改 JSON 解析，這裡要對應調整
video_file = generate_video_api(prompts['video'])



🎥 Replicate (Minimax) 正在生成影片...
✅ 原始影片已下載: raw_video.mp4


# ==========================================
# 步驟 4: 執行合併
# ==========================================

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips, vfx

def create_seamless_loop(video_path, output_path="seamless_loop.mp4", crossfade_duration=1.0):
    """
    將影片切半互換並 Crossfade，製作出視覺上的無縫循環素材。
    """
    print("🔄 正在處理無縫循環特效...")
    try:
        clip = VideoFileClip(video_path)
        duration = clip.duration

        # 1. 切成兩半
        clip1 = clip.subclip(0, duration/2)
        clip2 = clip.subclip(duration/2, duration)

        # 2. 交換順序 (Clip2 先，Clip1 後)，並在中間設定 Crossfade
        # padding=-crossfade_duration 會讓兩段影片重疊產生淡入淡出
        final_loop_clip = concatenate_videoclips([clip2, clip1], padding=-crossfade_duration, method="compose")

        # 儲存這個處理好的「無縫素材」
        final_loop_clip.write_videofile(output_path, codec="libx264", audio=False, verbose=False, logger=None)
        return output_path

    except Exception as e:
        print(f"❌ 循環處理失敗: {e}")
        return None

def assemble_video_loop(video_path, audio_path, output_name="final_video_loop.mp4"):
    if not video_path or not audio_path:
        return None

    print("🎬 正在合成最終循環影片...")
    try:
        # 1. 處理影片變成無縫素材
        seamless_clip_path = create_seamless_loop(video_path)
        seamless_clip = VideoFileClip(seamless_clip_path)

        # 2. 載入音樂
        audio_clip = AudioFileClip(audio_path)
        target_duration = audio_clip.duration  # 目標長度等於音樂長度 (例如 20秒)

        # 3. 讓影片無限循環直到填滿音樂長度
        # loop() 函式會自動重複播放影片
        final_video = seamless_clip.fx(vfx.loop, duration=target_duration)

        # 4. 合併音樂
        final_video = final_video.set_audio(audio_clip)

        # 5. 輸出
        final_video.write_videofile(output_name, fps=24, codec="libx264", audio_codec="aac")
        print(f"🎉 循環影片製作完成！檔案名稱: {output_name}")
        return output_name

    except Exception as e:
        print(f"❌ 合成失敗: {e}")
        return None

In [ ]:
# 授權Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 調整影片內容
import subprocess
import os
from moviepy.editor import VideoFileClip, AudioFileClip, vfx, concatenate_videoclips
from google.colab import files

def super_clean_video(input_file, output_file="super_fixed.mp4"):
    """
    強力修復模式：
    1. 強制指定 fps=24 (避免變動幀率導致的讀取錯誤)
    2. 強制指定像素格式 yuv420p (最相容格式)
    3. 重建檔案索引與標頭，解決 'bytes wanted but 0 bytes read' 錯誤
    """
    print(f"🔧 正在進行強力修復: {input_file} -> {output_file}")

    if not os.path.exists(input_file):
        print(f"❌ 找不到原始影片檔: {input_file}")
        return None

    # FFMPEG 指令
    command = [
        "ffmpeg",
        "-y",                 # 若檔案存在則覆蓋
        "-i", input_file,     # 輸入
        "-vf", "fps=24,format=yuv420p", # 強制規範化
        "-c:v", "libx264",    # 編碼器
        "-preset", "medium",  # 壓縮速度
        "-crf", "23",         # 畫質參數
        "-an",                # 移除音訊 (避免干擾)
        output_file           # 輸出
    ]

    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("✅ 強力修復完成！檔案結構已重建。")
        return output_file
    except subprocess.CalledProcessError as e:
        print(f"❌ FFMPEG 修復失敗: {e}")
        return None

def robust_assemble(video_path, audio_path):
    """
    穩健合成函式 (Ping-Pong 版)：
    1. 讀取影片時忽略音軌 (audio=False) 避免報錯
    2. 主動切除影片尾端 0.1 秒
    3. 使用【乒乓循環】(正播+倒播) 製作完美無縫影片
    """
    try:
        print("🎬 開始讀取媒體檔案...")
        clip = VideoFileClip(video_path, audio=False)
        music = AudioFileClip(audio_path)

        target_duration = music.duration
        print(f"🎵 音樂長度: {target_duration:.2f}s, 影片長度: {clip.duration:.2f}s")

        # 預防性切除尾端壞格
        if clip.duration > 0.1:
            clip = clip.subclip(0, clip.duration - 0.1)

        # ==========================================
        # 🔄 修改重點：使用 Ping-Pong (乒乓) 循環
        # ==========================================
        try:
            print("🔄 正在製作 Ping-Pong (來回播放) 無縫效果...")

            # 1. 建立倒轉片段
            reversed_clip = clip.fx(vfx.time_mirror)

            # 2. 接合：正向片段 + 反向片段
            # 這樣頭尾就會完美接合 (Start -> End -> End -> Start)
            seamless_clip = concatenate_videoclips([clip, reversed_clip])

            # 3. 讓這個「來回片段」無限循環直到填滿音樂長度
            final_clip = seamless_clip.fx(vfx.loop, duration=target_duration)

        except Exception as e:
            print(f"⚠️ Ping-Pong 處理失敗 ({e})，降級為簡單循環...")
            final_clip = clip.fx(vfx.loop, duration=target_duration)

        # 合成音訊
        final_clip = final_clip.set_audio(music)

        output_filename = "final_video_pingpong.mp4"
        print("💾 正在寫入最終檔案 (這可能需要幾秒鐘)...")
        final_clip.write_videofile(
            output_filename,
            codec="libx264",
            audio_codec="aac",
            fps=24,
            preset="medium",
            logger=None
        )
        print(f"🎉 成功！輸出檔案: {output_filename}")
        return output_filename

    except Exception as e:
        print(f"☠️ 合成過程發生錯誤: {e}")
        return None

# ==========================================
# 🚀 主執行區
# ==========================================

# 設定檔案名稱 (與你原本環境一致)
raw_video_name = "raw_video.mp4"
music_filename = "api_music.wav"

# 1. 執行修復
fixed_video = super_clean_video(raw_video_name)

# 2. 檢查檔案並執行合成
if fixed_video and os.path.exists(music_filename):
    print(f"✅ 確認音樂檔案存在: {music_filename}")

    # 執行 Ping-Pong 合成
    final_output = robust_assemble(fixed_video, music_filename)

    # 3. 自動下載
    if final_output:
        files.download(final_output)
else:
    print("❌ 無法執行：缺少必要的檔案 (raw_video.mp4 或 api_music.wav)")

🔧 正在進行強力修復: raw_video.mp4 -> super_fixed.mp4
✅ 強力修復完成！檔案結構已重建。
✅ 確認音樂檔案存在: api_music.wav
🎬 開始讀取媒體檔案...
🎵 音樂長度: 20.00s, 影片長度: 5.63s
🔄 正在製作 Ping-Pong (來回播放) 無縫效果...
💾 正在寫入最終檔案 (這可能需要幾秒鐘)...
🎉 成功！輸出檔案: final_video_pingpong.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>